# Tutorial of sampling and sampling-estimator

In this tutorial, we will see how to use the sampling and sampling-estimator functions of this challenge.  

In [1]:
# import 
from math import pi
from quri_parts.circuit import QuantumCircuit
from quri_parts.core.operator import Operator, pauli_label, PAULI_IDENTITY
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.state import GeneralCircuitQuantumState

## ChallengeSampling
In this challenge, all participants are required to use the provided sampling function. In order to this, it needs to import `ChallengeSampling`. The `ChallengeSampling` itself (defined in utills.challenge_2023.py) is a **Class** and you need to define a concrete instance `challenge_sampling`. From this instance you can construct a sampler and sampling_estimator. Here you can choose whether you will add the noise or not.

In [2]:
import sys

sys.path.append("../")
from utils.challenge_2023 import ChallengeSampling

# define challenge_sampling with or without noise
challenge_sampling = ChallengeSampling(noise=True)

## Prepare a circit

In [3]:
# circuit with 4 qubits
circuit = QuantumCircuit(4)
circuit.add_X_gate(0)
circuit.add_H_gate(1)
circuit.add_Y_gate(2)
circuit.add_CNOT_gate(1, 2)
circuit.add_RX_gate(3, pi/4)

# Sampler
When performing a sampling measurement for a circuit, you need a sampler function. By using the instance `challenge_sampling`, you can construct the sampler as follows.

In [4]:
# choose hardware type
hardware_type = "sc"

# create sampler
sampler = challenge_sampling.create_sampler(hardware_type)
sampling_result = sampler(circuit, n_shots=1000)
print(f"counts: {sampling_result}")

counts: Counter({5: 442, 3: 324, 13: 114, 11: 84, 1: 14, 7: 10, 2: 5, 4: 3, 9: 3, 15: 1})


A sampler receives two arguments: a circuit to be sampled and a number of repeated samplings (shots). The returned value is a mapping (dict) with the following keys and values:

- **Keys** Bitstrings of measurement outcomes, encoded as int. Each measured bit of each qubit are ordered from the least significant bit to the most significant bit. For example, when qubit 0 and qubit 2 are measured to be in  while the others are in , the bitstring is 0b0101.

- **Values** Counts of times for which each bitstring is measured. A sum of all counts is equal to the specified shots.

## Concurrent sampler
When performing a sampling measurement for some circuits, you can use a concurrent sampler. You can construct the concurrent sampler as same as the usual sampler.

In [5]:
# another circuit with 4 qubits
circuit2 = QuantumCircuit(4)
circuit2.add_T_gate(0)
circuit2.add_H_gate(1)
circuit2.add_RY_gate(2, 3*pi/4)
circuit2.add_CNOT_gate(1, 2)
circuit2.add_RZ_gate(3, pi)

# create concurrent sampler
concurrent_sampler = challenge_sampling.create_concurrent_sampler(hardware_type)
concurrent_counts = concurrent_sampler([(circuit, 1000), (circuit2, 1000)])

print(f"concurrent_counts: {concurrent_counts}")

concurrent_counts: [Counter({5: 479, 3: 375, 13: 61, 11: 47, 7: 12, 1: 9, 2: 7, 4: 5, 15: 4, 12: 1}), Counter({4: 533, 2: 287, 0: 105, 6: 75})]


# Sampling Estimator
In order to estimate an expectation value of operators involves operators and states, you need to use a sampling_estimator. You can construct the concurrent sampler as same as the usual sampler with some additional inputs.

In [6]:
# define an operator to be estimated
op = Operator({
    pauli_label("Z0"): 0.25,
    pauli_label("Z1 Z2"): 2.0,
    pauli_label("X1 X2"): 0.5 + 0.25j,
    pauli_label("Z1 Y3"): 1.0j,
    pauli_label("Z2 Y3"): 1.5 + 0.5j,
    pauli_label("X1 Y3"): 2.0j,
    PAULI_IDENTITY: 3.0,
})
print(op)

0.25*Z0 + 2.0*Z1 Z2 + (0.5+0.25j)*X1 X2 + 1j*Z1 Y3 + (1.5+0.5j)*Z2 Y3 + 2j*X1 Y3 + 3.0*I


## Pauli grouping
The operator is represented as a sum of Pauli operators. One of the ways to estimate expectation value of such an operator is to estimate expectation value of each Pauli term and then sum up them.

When estimating the Pauli terms, it is possible to measure multiple Pauli terms at once if they are commutable. The first step is thus to group the Pauli terms into several sets of commutable Pauli terms. This Pauli grouping is an important research subject in context of operator estimation.

One of the simplest Pauli grouping is bitwise grouping, where the groups are determined based on bitwise commutability of the Pauli terms. We can test the grouping as follows:

In [7]:
from quri_parts.core.operator.grouping import bitwise_pauli_grouping
pauli_sets = bitwise_pauli_grouping(op)
print(f"Number of groups: {len(pauli_sets)}")
for i, pauli_set in enumerate(pauli_sets):
    labels = ", ".join([str(pauli) for pauli in pauli_set])
    print(f"Group {i} contains: {labels}")

Number of groups: 5
Group 0 contains: X1 Y3
Group 1 contains: I
Group 2 contains: X1 X2
Group 3 contains: Z0, Z1 Z2
Group 4 contains: Z2 Y3, Z1 Y3


## Shot allocator
Another input necessary for estimation is PauliSamplingShotsAllocator: it specifies how total sampling shots should be allocated to measurement of each Pauli sets. There are several allocators available:

In [8]:
from quri_parts.core.sampling.shots_allocator import (
    create_equipartition_shots_allocator,
    create_proportional_shots_allocator,
    create_weighted_random_shots_allocator,
)
# Allocates shots equally among the Pauli sets
allocator = create_equipartition_shots_allocator()
# Allocates shots proportional to Pauli coefficients in the operator
allocator = create_proportional_shots_allocator()
# Allocates shots using random weights
allocator = create_weighted_random_shots_allocator(seed=777)

With these inputs, sampling estimation can be performed as follows:

In [9]:
shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement
n_shots = 10000

# create sampling estimator with above inputs
sampling_estimator = challenge_sampling.create_sampling_estimator(
    n_shots, measurement_factory, shots_allocator, hardware_type
)

# Since sampling_estimator measure for a quantum state, we prepare a circuit state.
circuit_state = GeneralCircuitQuantumState(4, circuit)

# this returns estimated value
estimated_value = sampling_estimator(op, circuit_state)
print(f"estimated_value :{estimated_value.value} ")

estimated_value :(1.5357999999999998+0.18739999999999996j) 


You can also construct the concurrent sampling_estimator as follows:

In [10]:
op2 = Operator({
    pauli_label("Z0 Y2"): 0.25 + 1.22j,
    pauli_label("Z1 Y2"): 1.4,
    pauli_label("X0 X2"): 0.2,
})

# create concurrent sampling estimator
concurrent_estimator = challenge_sampling.create_concurrent_sampling_estimator(
    n_shots, measurement_factory, shots_allocator, hardware_type
)

concurrent_v = concurrent_estimator([op, op2], [circuit_state])

print(f"concurrent_estimated_value :{concurrent_v[0].value, concurrent_v[1].value} ")

concurrent_estimated_value :((1.1236000000000002+0.1564j), (-0.11072-0.199104j)) 


## Parametric Sampling Estimator

In [11]:
from quri_parts.circuit import UnboundParametricQuantumCircuit
from quri_parts.core.state import ParametricCircuitQuantumState

p_circuit = UnboundParametricQuantumCircuit(4)
p_circuit.add_H_gate(0)
p_circuit.add_X_gate(1)
p_circuit.add_T_gate(3)
p_circuit.add_ParametricRX_gate(0)
p_circuit.add_ParametricRY_gate(1)
p_circuit.add_ParametricRZ_gate(3)

params = [1,2,3]
p_state = ParametricCircuitQuantumState(4, p_circuit)
parametric_sampling_estimator = challenge_sampling.create_parametric_sampling_estimator(
    n_shots,
    measurement_factory,
    shots_allocator,
    hardware_type,
    )
estimated_value = parametric_sampling_estimator(op, p_state, params)
print(f"estimated_value :{estimated_value.value} ")

# concurrent version

parametric_concurrent_sampling_estimator = challenge_sampling.create_concurrent_parametric_sampling_estimator(
    n_shots,
    measurement_factory,
    shots_allocator,
    hardware_type,
    )
params2 = [0.1, 0.2, 0.3]
estimated_values = parametric_concurrent_sampling_estimator(op, p_state, [params, params2])
print(f"concurrent_estimated_value :{estimated_values[0].value, estimated_values[1].value} ")

estimated_value :(3.7232+0.04119999999999996j) 
concurrent_estimated_value :((4.1698+0.22519999999999998j), (0.7596-0.555j)) 
